In [1]:
from json import dumps, loads
import time
import threading 
import ast
import unittest
import requests
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient
G = setup()[0]


In [2]:
global_client = ChainspaceClient(port=5000)


In [3]:
init_transaction = surge.init()
init_token1 = init_transaction['transaction']['outputs'][0]
init_token2 = init_transaction['transaction']['outputs'][1]
init_token3 = init_transaction['transaction']['outputs'][2]
init_token4 = init_transaction['transaction']['outputs'][3]
global_client.process_transaction(init_transaction)

inputs: ()
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 1}", "{\"type\": \"InitToken\", \"location\": 1}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {}}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


<Response [200]>

In [4]:
type(init_token1)

chainspacecontract.contract.ChainspaceObject

In [5]:
r1 = surge.SurgeClient(init_token1) # location=0
r2 = surge.SurgeClient(init_token2) # location=0
rep1 = surge.SREPClient() 

votes = (r1.cast_srep_vote(rep1.pub), r2.cast_srep_vote(rep1.pub))
rep1.create_srep_client(votes) # location=0
# r1.submit_bid('EBBuy', 50)
# r2.submit_bid('EBSell', 50)


inputs: ('{"type": "InitToken", "location": 0}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_616abd97764a2b04f8d9982a44cdcaddfd7b9eddc77c70c147f0d0ea00d67474"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd0206338ef472937be15a532823231e1883ccfb99dc32407623d662513c", "92c71d002b473adb80bb937aa77d787a4fe2bc8497c9da495e5a89fdb33922fa56c71d002b528b2fe9db704654cf53e8fd014d4682eaae59d4ac435dfe31a780a1"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0206338ef472937be15a532823231e1883ccfb99dc32407623d662513c\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0206338ef472937be15a532823231e1883ccfb99dc32407623d662513c\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0206338ef472937be15a532823231e1883ccfb99dc32407623d662513c\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surg

In [6]:
t1 = threading.Thread(target=r1.submit_bid, args=('EBBuy',50)) 
t2 = threading.Thread(target=r2.submit_bid, args=('EBSell',50)) 
t3 = threading.Thread(target=rep1.accept_bids) 

In [7]:
t1.start()
t2.start()
t3.start()
t1.join()
t2.join()
t3.join()

inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd0206338ef472937be15a532823231e1883ccfb99dc32407623d662513c"}',)
reference_inputs: ()
inputs: ('{"type": "EBToken", "location": 0, "pub": "c7220292cd02c9bd03430776847726217765bc859ed55c3c2fc8e6683c4dff9c70026b0fed"}',)
reference_inputs: ()
POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_b718f5be7a309b6f51d5ea46f3f78e193eae3840baa420cc16d77e779796391f"], "methodID": "submit_bid_proof", "parameters": ["EBBuy", "92c71d002b995853359deb975b660dfb3acff38769bd6e81a2746ea48725400e94c71d002b0ecaaccb28377984fe242b6678255e975c4c49174d2a378a6dc77ab4"], "outputs": ["{\"bid_type\": \"EBBuy\", \"type\": \"BidProof\", \"location\": 0, \"pub\": \"c7220292cd02c9bd0206338ef472937be15a532823231e1883ccfb99dc32407623d662513c\", \"quantity_sig\": \"92c71d002b4e3deeb97430579882eecf269935f3842a9c58ce957ddf65767e8efac71d002bc155ba874cfa3da5cddedac1a0fbcd622848322f1c273a9a994b19e8\"}", "{\"type\":

In [8]:
# '{}|{}'.format(50, loads(r1.ebtoken)['pub'])
# objs = global_client.get_objects({'location':0, 'type':'SurgeClient'})
# print (objs)
# bid_proofs = rep1.client.get_objects({ 'type':'EBBuy'})
# bid = ast.literal_eval(bid_proofs[0])
# bid['type']